## Day 8

[--- Day 8: Handheld Halting ---](https://adventofcode.com/2020/day/8)

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (`acc`, `jmp`, or `nop`) and an argument (a signed number like `+4` or `-20`).

- `acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, `acc +7` would increase the accumulator by `7`. The accumulator starts at `0`. After an acc instruction, the instruction immediately below it is executed next.
- `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, `jmp +2` would skip the next instruction, `jmp +1` would continue to the instruction immediately below it, and `jmp -20` would cause the instruction `20` lines above to be executed next.
- `nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:

    nop +0
    acc +1
    jmp +4
    acc +3
    jmp -3
    acc -99
    acc +1
    jmp -4
    acc +6
These instructions are visited in this order:

    nop +0  | 1
    acc +1  | 2, 8(!)
    jmp +4  | 3
    acc +3  | 6
    jmp -3  | 7
    acc -99 |
    acc +1  | 4
    jmp -4  | 5
    acc +6  |
First, the `nop +0` does nothing. Then, the accumulator is increased from `0` to `1` (`acc +1`) and `jmp +4` sets the next instruction to the other `acc +1` near the bottom. After it increases the accumulator from `1` to `2`, `jmp -4` executes, setting the next instruction to the only `acc +3`. It sets the accumulator to `5`, and `jmp -3` causes the program to continue back at the first `acc +1`.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is `5`.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

Your puzzle answer was `2080`.

`--- Part Two ---`

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a `jmp` is supposed to be a `nop`, or a `nop` is supposed to be a `jmp`. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one `jmp` or `nop`, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

    nop +0
    acc +1
    jmp +4
    acc +3
    jmp -3
    acc -99
    acc +1
    jmp -4
    acc +6
If you change the first instruction from `nop +0` to `jmp +0`, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the `jmp` instructions, the program will still eventually find another `jmp` instruction and loop forever.

However, if you change the second-to-last instruction (from `jmp -4` to `nop -4`), the program terminates! The instructions are visited in this order:

    nop +0  | 1
    acc +1  | 2
    jmp +4  | 3
    acc +3  |
    jmp -3  |
    acc -99 |
    acc +1  | 4
    nop -4  | 5
    acc +6  | 6
After the last instruction (`acc +6`), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (`acc +1`, `acc +1`, `acc +6`).

Fix the program so that it terminates normally by changing exactly one `jmp` (to `nop`) or `nop` (to `jmp`). What is the value of the accumulator after the program terminates?

Your puzzle answer was `2477`.

In [1]:
def day8():
    # A non-recursive solution
    
    index = 0
    index_list = []
    accumulator = 0
    data = open('./assets/advent2020_d8.txt').readlines()
    while True:
        action = data[index][:3]
        if action == 'acc':
            accumulator += int(data[index][4:])
            index += 1
        elif action == 'jmp':
            index = index + int(data[index][4:])
        else:
            index += 1
        if index in index_list:
            break
        else:
            index_list.append(index)
    part1 = accumulator

    # Part 2

    index = 0
    index_list = []
    accumulator = 0
    seq = 0
    seq_to_change = 0
    changed = False
    saved_accumlator = 0
    saved_index = 0
    while True:
        action = data[index][:3]
        if seq == seq_to_change and changed == False:
            if action == 'jmp':
                action = 'nop'
                changed = True
                saved_accumlator = accumulator
                saved_index = index
            elif action == 'nop':
                action = 'jmp'
                changed = True
                saved_accumlator = accumulator
                saved_index = index
            seq_to_change += 1
        seq += 1

        if action == 'acc':
            accumulator += int(data[index][4:])
            index += 1
        elif action == 'jmp':
            index = index + int(data[index][4:])
        else:
            index += 1

        if index >= len(data):
            break
        elif index in index_list:
            seq = seq_to_change - 1
            index = saved_index
            accumulator = saved_accumlator
            index_list = index_list[:seq]
            changed = False
        else:
            index_list.append(index)
    part2 = accumulator

    return part1, part2

In [2]:
# Display Results
part1,part2 = day8()
print(f'Part 1 Answer = {part1}')
print(f'Part 2 Answer = {part2}')

Part 1 Answer = 2080
Part 2 Answer = 2477


In [3]:
# Speed Test
%timeit day8()

6.35 ms ± 9.08 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# Tim Chen's recursive solution I liked
def part1func(s,a,i):
    if i in s: return a # infinite loop
    c,v = [y if x==0 else int(y) for x,y in enumerate(code[i].split())]
    return part1func(s+[i],(a+v if c=="acc" else a), i+(v if c=="jmp" else 1))

def part2func(s, a, i, f):
    if i >= len(code)-1: return a # the end!
    if i in s: return False # infinite loop
    c,v = [y if x==0 else int(y) for x,y in enumerate(code[i].split())]
    res = part2func(s+[i], (a+v if c=="acc" else a), i+(v if c=="jmp" else 1), f) # no flip
    if (c == "nop" or c == "jmp") and f == -1: # if flippable
        res = res or part2func(s+[i], a, i+(v if c=="nop" else 1), i) # flipped
    return res

def tim_day8():
    code = open("./assets/advent2020_d8.txt").read().split("\n")
    part1, part2 = part1func([],0,0), part2func([],0,0,-1)
    return part1, part2

part1,part2 = tim_day8()
print(f'Part 1 Answer = {part1}')
print(f'Part 2 Answer = {part2}')
%timeit tim_day8()

Part 1 Answer = 2080
Part 2 Answer = 2477
1.02 ms ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
